In [2]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using Plots

using Dates
using HDF5

In [10]:
function propagate(A, x0, steps)
    
    x = []
    push!(x, x0)

    @assert size(x0,1) == size(A,1) == size(A,2)

    for i=2:steps
        push!(x, A * x[end])
    end

    return x
end 

function bloch(ρ_list)
    # Pauli matricies
    σ = [ [0 1; 1 0], [0 -im; im 0], [1 0; 0 -1], [1 0; 0 1] ]

    bloch_vec = [
        convert.(Float64, [ ρ[1,2]+ρ[2,1],
                           (ρ[1,2]-ρ[2,1])*im,
                            ρ[1,1]-ρ[2,2]       ])
                for ρ in ρ_list]
    hcat(bloch_vec...)
end

bloch (generic function with 1 method)

In [3]:
function find_smallest_second_index(Y, threshold)
    # Find all indices where abs(x) > 1 + threshold
    indices = findall(x -> abs(x) > 1 + threshold, Y)

    # Extract the second (column) indices
    second_indices = [ind[2] for ind in indices]

    # Return the smallest second index if found, otherwise return nothing
    if !isempty(second_indices)
        return minimum(second_indices)
    else
        return nothing  # No elements found matching the condition
    end
end

find_smallest_second_index (generic function with 1 method)

In [19]:
function SB_Bloch4_DMD_physicality(γᵢ, time_multiplier, threshold)

    data_dir = "../DATA/"

    γᵢ = string(γᵢ)
    γᵢᶠ= parse(Float64, γᵢ)

    tᵍᵉˣʸ , ρᵍᵉˣʸ  = LiPoSID.read_GEXY_timeevolution(evol_data_file_name, γᵢ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    bᵉ = bloch(ρᵉ[1:lᵐᵃˣ])
    bᵍ = bloch(ρᵍ[1:lᵐᵃˣ])
    bˣ = bloch(ρˣ[1:lᵐᵃˣ])
    bʸ = bloch(ρʸ[1:lᵐᵃˣ])

    tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
    tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 

    t_list = [tᵉᶠ, tᵍᶠ, tˣᶠ, tʸᶠ]

    Δt = tᵉᶠ[2] - tᵉᶠ[1]

    t = t_list[argmin(length(tⁱ) for tⁱ in t_list)]
    
    ρˢᵇ = [ρᵉ[1:lᵐᵃˣ], ρᵍ[1:lᵐᵃˣ], ρˣ[1:lᵐᵃˣ], ρʸ[1:lᵐᵃˣ]]

    Yᵉ = [bᵉ; ones(lᵐᵃˣ)']
    Yᵍ = [bᵍ; ones(lᵐᵃˣ)']
    Yˣ = [bˣ; ones(lᵐᵃˣ)']
    Yʸ = [bʸ; ones(lᵐᵃˣ)']

    Yᵉ⁻ = Yᵉ[:,1:end-1]; Yᵉ⁺ = Yᵉ[:,2:end]
    Yᵍ⁻ = Yᵍ[:,1:end-1]; Yᵍ⁺ = Yᵍ[:,2:end]
    Yˣ⁻ = Yˣ[:,1:end-1]; Yˣ⁺ = Yˣ[:,2:end]
    Yʸ⁻ = Yᵉ[:,1:end-1]; Yʸ⁺ = Yᵉ[:,2:end]

    Y⁻ = hcat(Yᵉ⁻, Yᵍ⁻, Yˣ⁻, Yʸ⁻)
    Y⁺ = hcat(Yᵉ⁺, Yᵍ⁺, Yˣ⁺, Yʸ⁺)

    A⁴ᴰ = Y⁺ * pinv(Y⁻) # Direct DMD estimation

    A⁴ᴰc = log(A⁴ᴰ)/Δt
    
    bᵉ⁴ᴰ = hcat(propagate(A⁴ᴰ, [bᵉ[:,1]...,1], time_multiplier * lᵐᵃˣ)...)[1:3,:] 
    bᵍ⁴ᴰ = hcat(propagate(A⁴ᴰ, [bᵍ[:,1]...,1], time_multiplier * lᵐᵃˣ)...)[1:3,:]
    bˣ⁴ᴰ = hcat(propagate(A⁴ᴰ, [bˣ[:,1]...,1], time_multiplier * lᵐᵃˣ)...)[1:3,:]
    bʸ⁴ᴰ = hcat(propagate(A⁴ᴰ, [bʸ[:,1]...,1], time_multiplier * lᵐᵃˣ)...)[1:3,:]

    ind_e = find_smallest_second_index(bᵉ⁴ᴰ, threshold)
    ind_g = find_smallest_second_index(bᵍ⁴ᴰ, threshold)
    ind_x = find_smallest_second_index(bˣ⁴ᴰ, threshold)
    ind_y = find_smallest_second_index(bʸ⁴ᴰ, threshold)


    # Collect all values into a list and filter out the nothing values
    valid_indices = filter(!isnothing, [ind_e, ind_g, ind_x, ind_y])

    # Find the minimum if there are any valid values
    if !isempty(valid_indices)
        ind_min = minimum(valid_indices)
        println("Minimum valid index: ", ind_min)
    else
        ind_min = nothing
        println("All values are nothing, no minimum found.")
    end


    # Check if index_ERA is nothing
    if ind_min !== nothing
        # Perform the calculation if a valid index is found
        inverse_gamma_time = ind_min * Δt * γᵢᶠ
        println("inverse_gamma_time: ", inverse_gamma_time)
    else
        inverse_gamma_time = []
        # Handle the case where no valid index was found
        println("No valid index found, inverse_gamma_time cannot be calculated.")
    end
    
    return inverse_gamma_time

end

SB_Bloch4_DMD_physicality (generic function with 1 method)

In [27]:
evol_data_file_name = "DATA/ALL_GAMMAS_B4_D10.h5"
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]
γᵢ = γ[1]

"0.079477"

In [28]:
inverse_gamma_time_ExactDMD4 = SB_Bloch4_DMD_physicality(γᵢ, 2, 1e-6)

Minimum valid index: 1209
inverse_gamma_time: 1.9217538600000004


1.9217538600000004

In [30]:
println("Times in 1/γ when model predictions bacame unphysical:")

unphysical_times = []

for γᵢ in γ
    println("γ=",γᵢ)
    inverse_gamma_time = SB_Bloch4_DMD_physicality(γᵢ, 10, 1e-6)
    push!(unphysical_times, inverse_gamma_time)
    println(inverse_gamma_time)
end

Times in 1/γ when model predictions bacame unphysical:
γ=0.079477
Minimum valid index: 1209
inverse_gamma_time: 1.9217538600000004
1.9217538600000004
γ=0.25133
All values are nothing, no minimum found.
No valid index found, inverse_gamma_time cannot be calculated.
Any[]
γ=0.79477
All values are nothing, no minimum found.
No valid index found, inverse_gamma_time cannot be calculated.
Any[]
γ=2.5133
All values are nothing, no minimum found.
No valid index found, inverse_gamma_time cannot be calculated.
Any[]
γ=7.9477
All values are nothing, no minimum found.
No valid index found, inverse_gamma_time cannot be calculated.
Any[]
γ=25.133
All values are nothing, no minimum found.
No valid index found, inverse_gamma_time cannot be calculated.
Any[]
γ=79.477
All values are nothing, no minimum found.
No valid index found, inverse_gamma_time cannot be calculated.
Any[]
γ=251.33
Minimum valid index: 821
inverse_gamma_time: 10.317096500000002
10.317096500000002


In [31]:
unphysical_times

8-element Vector{Any}:
  1.9217538600000004
   Any[]
   Any[]
   Any[]
   Any[]
   Any[]
   Any[]
 10.317096500000002